In [7]:
import threading
import time

def run(n):
    print("task", n,threading.current_thread())
    time.sleep(1)
    print('3s')
    time.sleep(1)
    print('2s')
    time.sleep(1)
    print('1s')
    
start_time = time.time()

t_obj = []

for i in range(3):
    t = threading.Thread(target=run, args=("t-%s" % i,))
    t.start()
    t_obj.append(t)

for tmp in t_obj:
    t.join()

print('cost:',time.time() - start_time)
print(threading.current_thread())

task t-0 <Thread(Thread-19, started 123145593528320)>
task t-1task <Thread(Thread-20, started 123145598783488)>
cost: t-2 <Thread(Thread-21, started 123145604038656)>
 0.015045166015625
<_MainThread(MainThread, started 4579204416)>
3s
3s3s

2s
2s2s

1s
1s1s



In [3]:
import threading
import time

def run(n):
    lock.acquire()
    global num
    num += 1
    lock.release()
lock = threading.Lock()
num = 0
t_obj = [] 

for i in range(1000):
    t = threading.Thread(target=run, args=("t-%s" % i,))
    t.start()
    t_obj.append(t)

for t in t_obj:
    t.join()

print("num:", num)

num: 1000


死锁，所谓死锁：是指两个或两个以上的进程或线程在执行过程中，因争夺资源而造成的一种互相等待的现象，若无外力作用，它们都将无法推进下去。此时称系统处于死锁状态或系统产生了死锁，这些永远在互相等待的进程成为死锁进程。

In [14]:
import threading
import time
mutexA=threading.Lock()
mutexB=threading.Lock()
 
class MyThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
    def run(self):
        self.fun1()
        self.fun2()
 
    def fun1(self):
        mutexA.acquire()
        print("I am %s,get res:%s---%s"%(self.name,"ResA",time.time()))
 
        mutexB.acquire()
        print("I am %s,get res:%s---%s"%(self.name,"ResB",time.time()))
        mutexB.release()
 
        mutexA.release()
 
 
    def fun2(self):
        mutexB.acquire()
        print("I am %s,get res:%s---%s"%(self.name,"ResB",time.time()))
        time.sleep(0.2)
        mutexA.acquire()
        print("I am %s,get res:%s---%s"%(self.name,"ResA",time.time()))
        mutexA.release()
        mutexA.release()
        
if __name__ == '__main__':
    print("start------------%s",time.time())
 
    for i in range(0,10):
        my_thread=MyThread()
        my_thread.start()

start------------%s 1570431671.7803662
I am Thread-3094,get res:ResA---1570431671.7808309
I am Thread-3094,get res:ResB---1570431671.781753
I am Thread-3094,get res:ResB---1570431671.7817888
I am Thread-3095,get res:ResA---1570431671.781847


递归锁，在python中为了支持在同一线程中多次请求同一资源，python提供了可重入锁RLock。这个RLock内部维护着一个Lock和一个counter变量，counter记录了acquire的次数，从而使得资源可以被多次require。直到一个线程所有的acquire都被release，其他的线程才能获得资源。

In [15]:
import threading
import time
 
Rlock=threading.RLock()
class MyThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
 
    def run(self):
        self.fun1()
        self.fun2()
 
    def fun1(self):
        Rlock.acquire() #如果锁被占用，则阻塞在这里，等待锁的释放
        print("I am %s,get res:%s---%s"%(self.name,"ResA",time.time()))
 
        Rlock.acquire()  #count=2
        print("I am %s,get res:%s---%s"%(self.name,"ResB",time.time()))
 
        Rlock.release()   #count-1
        Rlock.release()   #count-1=0
 
 
 
    def fun2(self):
        Rlock.acquire()  #count=1
        print("I am %s,get res:%s---%s"%(self.name,"ResB",time.time()))
        time.sleep(0.2)
 
        Rlock.acquire()  #count=2
        print("I am %s,get res:%s---%s"%(self.name,"ResA",time.time()))
        Rlock.release()  #coun-1
        Rlock.release()  #count-1

        
if __name__ == '__main__':
    print("start-----------%s"%time.time())
 
    for i in range(0,10):
        my_thread=MyThread()
        my_thread.start()

start-----------1570431695.364209
I am Thread-3104,get res:ResA---1570431695.3650858
I am Thread-3104,get res:ResB---1570431695.367688
I am Thread-3104,get res:ResB---1570431695.367748
I am Thread-3104,get res:ResA---1570431695.571456
I am Thread-3106,get res:ResA---1570431695.5723321
I am Thread-3106,get res:ResB---1570431695.572829
I am Thread-3106,get res:ResB---1570431695.572864
I am Thread-3106,get res:ResA---1570431695.777215
I am Thread-3107,get res:ResA---1570431695.777784
I am Thread-3107,get res:ResB---1570431695.7781131
I am Thread-3107,get res:ResB---1570431695.778157
I am Thread-3107,get res:ResA---1570431695.9824588
I am Thread-3109,get res:ResA---1570431695.982678
I am Thread-3109,get res:ResB---1570431695.983159
I am Thread-3109,get res:ResB---1570431695.9831839
I am Thread-3109,get res:ResA---1570431696.1877902
I am Thread-3111,get res:ResA---1570431696.188295
I am Thread-3111,get res:ResB---1570431696.1889
I am Thread-3111,get res:ResB---1570431696.188961
I am Thread-

信号量（BoundedSemaphore类）

互斥锁同时只允许一个线程更改数据，而Semaphore是同时允许一定数量的线程更改数据 ，比如厕所有3个坑，那最多只允许3个人上厕所，后面的人只能等里面有人出来了才能再进去。

In [20]:
import threading
import time


def run(n):
    semaphore.acquire()   #加锁
    time.sleep(1)
    print("run the thread:%s\n" % n)
    semaphore.release()     #释放


num = 0
semaphore = threading.BoundedSemaphore(5)  # 最多允许5个线程同时运行

for i in range(22):
    t = threading.Thread(target=run, args=("t-%s" % i,))
    t.start()

run the thread:t-1
run the thread:t-2
run the thread:t-3
run the thread:t-0
run the thread:t-4





run the thread:t-6
run the thread:t-5
run the thread:t-7
run the thread:t-9
run the thread:t-8





run the thread:t-13
run the thread:t-10
run the thread:t-12
run the thread:t-11
run the thread:t-14





run the thread:t-18
run the thread:t-16
run the thread:t-17
run the thread:t-19
run the thread:t-15





run the thread:t-20

run the thread:t-21



事件（Event类）
python线程的事件用于主线程控制其他线程的执行，事件是一个简单的线程同步对象，其主要提供以下几个方法：
事件处理的机制：全局定义了一个“Flag”，当flag值为“False”，那么event.wait()就会阻塞，当flag值为“True”，那么event.wait()便不再阻塞。

In [21]:
#利用Event类模拟红绿灯
import threading
import time

event = threading.Event()


def lighter():
    count = 0
    event.set()     #初始值为绿灯
    while True:
        if 5 < count <=10 :
            event.clear()  # 红灯，清除标志位
            print("\33[41;1mred light is on...\033[0m")
        elif count > 10:
            event.set()  # 绿灯，设置标志位
            count = 0
        else:
            print("\33[42;1mgreen light is on...\033[0m")

        time.sleep(1)
        count += 1

def car(name):
    while True:
        if event.is_set():      #判断是否设置了标志位
            print("[%s] running..."%name)
            time.sleep(1)
        else:
            print("[%s] sees red light,waiting..."%name)
            event.wait()
            print("[%s] green light is on,start going..."%name)

light = threading.Thread(target=lighter,)
light.start()

car = threading.Thread(target=car,args=("MINI",))
car.start()

green light is on...
[MINI] running...
green light is on...
[MINI] running...
green light is on...
[MINI] running...
green light is on...
[MINI] running...
green light is on...
[MINI] running...
green light is on...
[MINI] running...
red light is on...
[MINI] sees red light,waiting...
red light is on...
red light is on...
red light is on...
red light is on...
[MINI] green light is on,start going...
[MINI] running...
green light is on...[MINI] running...

[MINI] running...green light is on...

[MINI] running...green light is on...

green light is on...[MINI] running...

green light is on...[MINI] running...

red light is on...[MINI] sees red light,waiting...

red light is on...
red light is on...
red light is on...
red light is on...
[MINI] green light is on,start going...
[MINI] running...
[MINI] running...green light is on...

[MINI] running...green light is on...

[MINI] running...green light is on...

[MINI] running...green light is on...

green light is on...[MINI] running...

[MIN

定时器（Timer类）

定时器，指定n秒后执行某操作

In [1]:
from threading import Timer
 
def hello():
    print("hello, world")

t = Timer(1, hello)
t.start()  # after 1 seconds, "hello, world" will be printed

hello, world


In [ ]:
import os